In [1]:
import keras 
import numpy as np

Using TensorFlow backend.
c:\users\satwik\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\satwik\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\satwik\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.in

In [2]:
text=open("text.txt").read().lower()
print("Corpus length",len(text))

Corpus length 35302


In [3]:
maxlen=60
step=3
sentences = []
next_chars = []

for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print("Number of sequences: ",len(sentences))

chars=sorted(list(set(text)))
print("Uniques characters: ",len(chars))
char_indices= dict((char,chars.index(char)) for char in chars)

print("Vectorization...")
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool) 
for i, sentence in enumerate(sentences): 
    for t, char in enumerate(sentence): 
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences:  11748
Uniques characters:  49
Vectorization...


In [4]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(chars))))
model.add(layers.Dense(len(chars),activation="softmax"))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss="categorical_crossentropy",optimizer=optimizer)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
def sample(preds, temperature=1.0): 
    preds = np.asarray(preds).astype('float64') 
    preds = np.log(preds) / temperature 
    exp_preds = np.exp(preds) 
    preds = exp_preds / np.sum(exp_preds) 
    probas = np.random.multinomial(1, preds, 1) 
    return np.argmax(probas)

In [ ]:
import random 
import sys

for epoch in range(1, 60): 
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1) 
    start_index = random.randint(0, len(text) - maxlen - 1) 
    generated_text = text[start_index: start_index + maxlen] 
    print('--- Generating with seed: "' + generated_text + '"')
    for temperature in [0.2, 0.5, 1.0, 1.2]: 
        print('------ temperature:', temperature) 
        sys.stdout.write(generated_text)
    for i in range(400): 
        sampled = np.zeros((1, maxlen, len(chars))) 
        for t, char in enumerate(generated_text): 
            sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0] 
            next_index = sample(preds, temperature) 
            next_char = chars[next_index]
            generated_text += next_char 
            generated_text = generated_text[1:]
    sys.stdout.write(next_char)

epoch 1
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
11748/11748 [==============================] - 17s 1ms/step - loss: 2.8399
--- Generating with seed: "a book.
i'm gonna get dress.
we isolate the disturbances
to "
------ temperature: 0.2
a book.
i'm gonna get dress.
we isolate the disturbances
to ------ temperature: 0.5
a book.
i'm gonna get dress.
we isolate the disturbances
to ------ temperature: 1.0
a book.
i'm gonna get dress.
we isolate the disturbances
to ------ temperature: 1.2
a book.
i'm gonna get dress.
we isolate the disturbances
to  epoch 2
Epoch 1/1
11748/11748 [==============================] - 13s 1ms/step - loss: 2.3040
--- Generating with seed: "
be leaving behind.
mommy?
it's over.
i'm so sorry!
i love y"
------ temperature: 0.2

be leaving behind.
mommy?
it's over.
i'm so sorry!
i love y------ temperature: 0.5

be leaving behind.
mommy?
it's over.
i'm so sorry!
i love y------ temperature: 1.0

be leaving behind.
mommy?
it's over.
i'm so sorry!
i love 